In [1]:
import os
import json5
import json
import math
import pandas as pd
import numpy as np
from datasets import load_dataset,Dataset
from collections import Counter

In [2]:
origin_xlsx = '/mnt/users/LLMa/ChatGLM-6B/data/MRA/头MRA数据-24个中心所有数据汇总-to深睿-1101.xlsx'
train_xlsx =  '/mnt/users/LLMa/ChatGLM-6B/data/MRA/东总血管疾病stl信息统计_20230220_3222例.xlsx'
val_xlsx = '/mnt/users/LLMa/ChatGLM-6B/data/MRA/东总血管疾病stl信息统计_1172例.xlsx'

In [3]:
origin_df = pd.read_excel(origin_xlsx,converters={'ID': str})
# train_df = pd.read_excel(train_xlsx,converters={'ID': str})
# val_df = pd.read_excel(val_xlsx,converters={'ID': str})

percentile = 0.85
# origin_df['ID'] = origin_df['ID'].astype(str)
# train_df['ID'] = train_df['ID'].astype(str)
# val_df['ID'] = val_df['ID'].astype(str)

In [4]:
origin_df['MR所见'].replace('', np.nan, inplace=True)
origin_df.dropna(subset=['MR所见'], inplace=True)

origin_df['MR报告'].replace('', np.nan, inplace=True)
origin_df.dropna(subset=['MR报告'], inplace=True)

In [5]:
# origin_df['MR所见'] = origin_df['MR所见'].str.strip()
origin_df.replace('\s+','',regex=True,inplace=True) 
origin_df.replace('_x000D_','',regex=True,inplace=True)

origin_df.replace('[2-9][、\.](?=[\u4e00-\u9fa5])','；',regex=True,inplace=True) 
origin_df.replace('[1][、\.](?=[\u4e00-\u9fa5])','',regex=True,inplace=True) 
origin_df.replace('；；','；',regex=True,inplace=True) 
origin_df.replace('。；','；',regex=True,inplace=True) 
origin_df.replace(';；','；',regex=True,inplace=True) 

# origin_df.replace('[1-9\.\u4e00-\u9fa5]',';',regex=True,inplace=True) 
idx = int(len(origin_df)*percentile)
train_part = origin_df[:idx]
val_part = origin_df[idx:]


In [7]:
report_lens=[]
conlusion_lens =[]
for idx,row  in origin_df.iterrows():
    if not pd.isna(row['MR所见']):
        report_lens.append(len(row['MR所见']))
    if not pd.isna(row['MR报告']):
        conlusion_lens.append(len(row['MR报告']))
print(Counter(report_lens).most_common())
print(Counter(conlusion_lens).most_common())
print(np.sum(np.asarray(report_lens)>512))
print(np.sum(np.asarray(conlusion_lens)>512))

[(61, 76), (258, 73), (222, 63), (248, 58), (244, 57), (237, 56), (256, 56), (261, 56), (273, 55), (294, 55), (271, 54), (289, 54), (281, 54), (284, 54), (239, 54), (266, 54), (286, 54), (259, 53), (242, 53), (232, 53), (252, 53), (220, 53), (208, 52), (210, 52), (245, 52), (224, 52), (225, 52), (228, 52), (236, 51), (267, 51), (249, 51), (265, 51), (270, 51), (246, 50), (218, 50), (229, 50), (240, 50), (234, 50), (231, 50), (219, 50), (214, 50), (274, 49), (238, 49), (254, 49), (233, 49), (283, 49), (251, 49), (272, 48), (262, 48), (227, 48), (264, 48), (290, 47), (263, 47), (230, 47), (275, 46), (255, 46), (280, 46), (241, 46), (216, 46), (247, 46), (277, 45), (213, 45), (260, 45), (253, 45), (223, 45), (235, 45), (301, 45), (292, 44), (257, 44), (288, 44), (221, 44), (291, 44), (209, 43), (302, 43), (226, 43), (287, 43), (269, 43), (201, 42), (295, 42), (268, 42), (211, 41), (194, 41), (243, 41), (217, 41), (207, 41), (204, 40), (297, 40), (197, 40), (212, 40), (250, 39), (189, 39),

In [105]:

train_pairs = {"Input":list(train_part['MR所见']),
         "Output":list(train_part['MR报告'])}
train_data = Dataset.from_dict(train_pairs)
print(len(train_data))
# train_data.to_csv('/mnt/users/LLMa/ChatGLM-6B/data/MRA/train_report2conclusion.csv')
train_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/MRA/train_conclusion.json',force_ascii=False)
    

9295


Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

7429096

In [106]:

val_pairs = {"Input":list(val_part['MR所见']),
         "Output":list(val_part['MR报告'])}
val_data = Dataset.from_dict(val_pairs)
print(len(val_data))

val_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/MRA/val_conclusion.json',force_ascii=False)

1641


Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1658387